In [1]:
from glob import glob
from tqdm import tqdm
from pathlib import Path
import zipfile
import sys
import dotenv
import os

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import tensorflow as tf
tf.compat.v1.enable_eager_execution()

dotenv.load_dotenv(override=True)
WORK = os.getenv('WORK')
sys.path.insert(0, f'{WORK}/ADA_Project')
sys.path.insert(0, f'{WORK}/ADA_Project/StyleGAN2-ADA')

import DeepFAMS

WORK, PROJ_DIR = DeepFAMS.utils.set_env()

Tensorflow version: 1.14.0


In [2]:
RAW_IMGS_DIR, RESIZED_IMGS_DIR, DATA_CUSTOM_DIR, TRAIN_RUNS_DIR = DeepFAMS.utils.return_dirs(
    PROJ_DIR, 'FFHQ')

In [4]:
# ! $WORK/.conda/envs/ada-env/bin/kaggle datasets download -d \
#     arnaud58/flickrfaceshq-dataset-ffhq -p $WORK/ADA_Project/datasets

In [5]:
# downloaded_file = f'{Path(RAW_IMGS_DIR).parent}/flickrfaceshq-dataset-ffhq.zip'

# with zipfile.ZipFile(downloaded_file) as zf:    
#     for member in tqdm(zf.infolist(), desc='Extracting'):
#         try:
#             zf.extract(member, RAW_IMGS_DIR)
#         except zipfile.error as e:
#             pass

In [6]:
raw_imgs = glob(f'{RAW_IMGS_DIR}/*')
print(len(raw_imgs))

52001


In [7]:
# for x in tqdm(raw_imgs):
#     DeepFAMS.preprocessing.resize_imgs(x, (256, 256), RESIZED_IMGS_DIR)

In [8]:
print(f'Raw: {len(raw_imgs)}, Resized: {len(glob(f"{RESIZED_IMGS_DIR}/*"))}')

Raw: 52001, Resized: 52001


In [14]:
# DeepFAMS.preprocessing.tf_record_exporter(
#     tfrecord_dir=DATA_CUSTOM_DIR, image_dir=RESIZED_IMGS_DIR, shuffle=1)

Loading images from "/work/chaselab/malyetama/ADA_Project/datasets/FFHQ_resized"
Creating dataset "/work/chaselab/malyetama/ADA_Project/datasets/FFHQ_custom_30K"
Added 30000 images.                     


In [16]:
DeepFAMS.utils.execute('nvidia-smi')

Sun Apr 18 19:24:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.67       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100S-PCI...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   71C    P0   174W / 250W |   9095MiB / 32510MiB |     85%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100S-PCI...  On   | 00000000:86:00.0 Off |                    0 |
| N/A   

In [17]:
DeepFAMS.utils.executePopen(f'''#!/bin/bash
module load anaconda
module load compiler/gcc/4.7
module load cuda
$WORK/.conda/envs/ada-env/bin/python $WORK/ADA_Project/StyleGAN2-ada/train.py \
--outdir={TRAIN_RUNS_DIR} \
--gpus=2 \
--data={DATA_CUSTOM_DIR} \
--snap=1 \
--kimg=1''', PROJ_DIR)



Training options:

{

  "G_args": {

    "func_name": "training.networks.G_main",

    "fmap_base": 8192,

    "fmap_max": 512,

    "mapping_layers": 2,

    "num_fp16_res": 4,

    "conv_clamp": 256

  },

  "D_args": {

    "func_name": "training.networks.D_main",

    "mbstd_group_size": 4,

    "fmap_base": 8192,

    "fmap_max": 512,

    "num_fp16_res": 4,

    "conv_clamp": 256

  },

  "G_opt_args": {

    "beta1": 0.0,

    "beta2": 0.99,

    "learning_rate": 0.0025

  },

  "D_opt_args": {

    "beta1": 0.0,

    "beta2": 0.99,

    "learning_rate": 0.0025

  },

  "loss_args": {

    "func_name": "training.loss.stylegan2",

    "r1_gamma": 0.4096

  },

  "augment_args": {

    "class_name": "training.augment.AdaptiveAugment",

    "tune_heuristic": "rt",

    "tune_target": 0.6,

    "apply_func": "training.augment.augment_pipeline",

    "apply_args": {

      "xflip": 1,

      "rotate90": 1,

      "xint": 1,

      "scale": 1,

      "rotate": 1,

      "aniso": 1,


Calculating real image statistics for fid50k_full...




In [18]:
for num in range(-1, -10, -1):
    files = DeepFAMS.utils.last_snap(num, TRAIN_RUNS_DIR)
    if files != []:
        break

latest_snap = sorted(files)[-1]
print(latest_snap)

/work/chaselab/malyetama/ADA_Project/training_runs/FFHQ_training-runs_30K/00000-FFHQ_custom_30K-auto2-kimg1/network-snapshot-000000.pkl


In [19]:
run_desc, training_options = DeepFAMS.setup_training_options(
    gpus       = 2,
    snap       = 30,
    data       = DATA_CUSTOM_DIR,
    resume     = latest_snap
)

In [20]:
DeepFAMS.RunTraining(outdir=TRAIN_RUNS_DIR, seed=1000,
             dry_run=True, run_desc=run_desc, training_options=training_options)


Training options:
{
  "G_args": {
    "func_name": "training.networks.G_main",
    "fmap_base": 8192,
    "fmap_max": 512,
    "mapping_layers": 2,
    "num_fp16_res": 4,
    "conv_clamp": 256
  },
  "D_args": {
    "func_name": "training.networks.D_main",
    "mbstd_group_size": 4,
    "fmap_base": 8192,
    "fmap_max": 512,
    "num_fp16_res": 4,
    "conv_clamp": 256
  },
  "G_opt_args": {
    "beta1": 0.0,
    "beta2": 0.99,
    "learning_rate": 0.0025
  },
  "D_opt_args": {
    "beta1": 0.0,
    "beta2": 0.99,
    "learning_rate": 0.0025
  },
  "loss_args": {
    "func_name": "training.loss.stylegan2",
    "r1_gamma": 0.4096
  },
  "augment_args": {
    "class_name": "training.augment.AdaptiveAugment",
    "tune_heuristic": "rt",
    "tune_target": 0.6,
    "apply_func": "training.augment.augment_pipeline",
    "apply_args": {
      "xflip": 1,
      "rotate90": 1,
      "xint": 1,
      "scale": 1,
      "rotate": 1,
      "aniso": 1,
      "xfrac": 1,
      "brightness": 1,
   

In [ ]:
tf.compat.v1.disable_eager_execution()
DeepFAMS.RunTraining(outdir=TRAIN_RUNS_DIR, seed=1000,
             dry_run=False, run_desc=run_desc, training_options=training_options)